In [1]:
from pyspark.sql import SparkSession
from vars import *
from datetime import date
from functions import loadConfigs
import os
from pyspark.sql.functions import explode
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from delta import *

# minio credentials
minio_endpoint = "http://minio:9000"
minio_access_key = "iQekrU3nA33aKktu2GMo"
minio_secret_key = "1OGQ3m1lWEZVQ31xKfPCKmZVFVb2lXwCorjZr1d1"
minio_region = "us-east-1"
minio_bucket = "reddit-datalake"

builder = loadConfigs(SparkSession.builder)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
from columns import gildings

In [3]:
# today = date.today().strftime('%Y%m%d')
today = 20230610

output_folder = "gildings"
output_file = "gildings"

df_raw = spark.read.option("header", "true") \
    .json(f"s3a://{minio_bucket}/bronze/popular_{today}.json")

df_raw = df_raw.select(explode(df_raw.data.children.data).alias("data"))
df_raw = df_raw.select("data.*")

df_gildings = df_raw.select("id", "author", "gilded", "gildings", "is_video", "score", "subreddit_id", "subreddit") \
                      .withColumnRenamed("id", "post_id")

df_gildings = df_gildings.select("*", "gildings.*")

if "gid_1" not in df_gildings.columns:
    df_gildings = df_gildings.withColumn("gid_1", lit(None).cast(IntegerType()))

df_renamed = df_gildings.withColumnRenamed("gid_1", "gild_silver") \
                        .withColumnRenamed("gid_2", "gild_gold") \
                        .withColumnRenamed("gid_3", "gild_platinum")

df_final = df_renamed.dropDuplicates()

df_final = df_final.withColumn("dateid", lit(today))

df_final = df_final.select(gildings)

# df_final.write.mode("overwrite").parquet(f"s3a://{minio_bucket}/silver/{output_folder}/{output_file}_{today}")

In [4]:
df_final.show()

+-------+-------------------+------+------------------+-----------+---------+-------------+--------+-----+------------+--------------------+--------+
|post_id|             author|gilded|          gildings|gild_silver|gild_gold|gild_platinum|is_video|score|subreddit_id|           subreddit|  dateid|
+-------+-------------------+------+------------------+-----------+---------+-------------+--------+-----+------------+--------------------+--------+
|145hgit|TheUndeadGunslinger|     1|      {1, 1, null}|          1|        1|         null|   false|20451|    t5_2qh03|              gaming|20230610|
|145hijg|     yycsoftwaredev|     0|{null, null, null}|       null|     null|         null|   false| 3123|    t5_2qh68|              canada|20230610|
|145uv3p|            esberat|     0|{null, null, null}|       null|     null|         null|    true| 5147|    t5_qir9n|               HolUp|20230610|
|145owhy|        BigHoneyBot|     0|{null, null, null}|       null|     null|         null|   false|

In [5]:
df_final.write.format("delta").mode("overwrite").save(f"s3a://{minio_bucket}/silver/{output_folder}/{output_file}_{today}")